In [ ]:
df=pd.read_csv('flipkart_com-ecommerce_sample_1050.csv')

# Récupération des catégories
def extract_first_category(category_string):
    # Convertir la chaîne de caractères en liste
    category_list = ast.literal_eval(category_string)
    
    # Sélectionner la première valeur de la liste
    full_category = category_list[0]
    
    # Sélectionner la première partie avant '>>'
    first_category = full_category.split('>>')[0].strip()
    
    return first_category

df['main_category'] = df['product_category_tree'].apply(extract_first_category)
list_labels = df['main_category'].unique().tolist()
df.head(1)
df.shape
list_labels


path = r"C:\Users\mauge\Openclassrooms\Projet 7 - Git repository\Veille\Data\Images"

data_path = glob(path + '/*.jp*')

def data_fct(path) :
    list_photos = [file for file in path]
    print(len(list_photos))
    data = pd.DataFrame()
    data["image_path"] = list_photos
    return data

data = data_fct(data_path)

data.head(5)
data['file_id'] = data['image_path'].apply(lambda x: x.split('\\')[-1].split('.')[0])
df.rename(columns={'uniq_id': 'file_id'}, inplace=True)
data = pd.merge(data, df, on='file_id', how='left')

In [ ]:
def image_prep_fct(data) :
    prepared_images = []
    for image_num in range(len(data['image_path'])) :
        img = (load_img(
            data['image_path'][image_num],
            target_size=(224, 224)))
        img = img_to_array(img)
        img = img.reshape((img.shape[0], img.shape[1], img.shape[2]))
        img = preprocess_input(img)
        prepared_images.append(img)
        prepared_images_np = np.array(prepared_images)
    return prepared_images_np
    
images_np = image_prep_fct(data)
print(images_np.shape)



# Initialiser le LabelEncoder
label_encoder = LabelEncoder()
# Convertir les étiquettes de catégorie en entiers
integer_encoded = label_encoder.fit_transform(data['main_category'])
# Convertir les étiquettes en format one-hot
y_labels_one_hot = to_categorical(integer_encoded, num_classes=7)
X = images_np
y = y_labels_one_hot
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train.shape